<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/4_%E7%BB%B4%E5%BA%A6%E5%8F%98%E6%8D%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.Tensor维度变换

view/reshape 变形

squeeze/unsqueeze 删减维度，增加维度

expand/repeat 维度扩展/复制

t/transpose/permute 转置/维度交换/维度重排

Broadcast 自动扩展维度


### view/reshape 变形
view/reshape变形，功能一样，前提是保证整个tensor的numel()不变，即元素个数不变：prod(a.size)=prod(a`.size)。不变的数据本身，变的是对数据的理解。

In [0]:
import torch
import numpy as np

In [0]:
a=torch.rand(4,1,28,28)
a.shape

torch.Size([4, 1, 28, 28])

In [0]:
a.view(4,28*28) #变形为[4,784]，适合全连接层的输入（要输入向量）

tensor([[0.7606, 0.9825, 0.2617,  ..., 0.9743, 0.5972, 0.9635],
        [0.4072, 0.3821, 0.4051,  ..., 0.6794, 0.3975, 0.8813],
        [0.8691, 0.4486, 0.9818,  ..., 0.3329, 0.8123, 0.6995],
        [0.1522, 0.6267, 0.3005,  ..., 0.1892, 0.1495, 0.6522]])

In [0]:
a.view(4*28,28).shape #只关注w

torch.Size([112, 28])

In [0]:
a.view(4*1,28,28).shape #只关注feature map，不关注哪个图片哪个通道

torch.Size([4, 28, 28])

数据的存储/维度顺序非常重要，需要时刻记住维度顺序信息，否则会数据污染

In [0]:
b=a.view(4,784)
b.view(4,28,28,1).shape #数据恢复错了，应该是b,c,h,w [4,1,28,28]

torch.Size([4, 28, 28, 1])

### unsqueeze 增加维度
插入的维度是0字节，没有改变数据本身，只是改变了数据的理解方式


In [0]:
a=torch.rand(4,1,28,28)
a.shape

torch.Size([4, 1, 28, 28])

In [0]:
a.unsqueeze(0).shape #在某位置插入一个维度

torch.Size([1, 4, 1, 28, 28])

In [0]:
a.unsqueeze(3).shape

torch.Size([4, 1, 28, 1, 28])

实用举例1

In [0]:
b=torch.tensor([1.2,3.4]) #1维
b

tensor([1.2000, 3.4000])

In [0]:
b.shape

torch.Size([2])

In [0]:
c=b.unsqueeze(1) #2维
c

tensor([[1.2000],
        [3.4000]])

In [0]:
c.shape

torch.Size([2, 1])

实用举例2

In [0]:
f=torch.rand(1,32,1,1)
bias=torch.rand(32) #偏差，想要每个channel上的所有像素增加一个偏置
bias.shape

torch.Size([32])

In [0]:
bias=bias.unsqueeze(0).unsqueeze(2).unsqueeze(3) #使它和f的维度结构一样，才能相加
bias.shape #这种连续插入，要用最后的位置索引

torch.Size([1, 32, 1, 1])

### squeeze 删减维度

In [0]:
a=torch.rand(1,32,1,1)
a.shape

torch.Size([1, 32, 1, 1])

In [0]:
a.squeeze().shape #不加参数，默认删除所有能去掉的的维度

torch.Size([32])

In [0]:
a.squeeze(0).shape

torch.Size([32, 1, 1])

In [0]:
a.squeeze(1).shape #第1维的shape不为1，所以挤压不掉

torch.Size([1, 32, 1, 1])

### expand 某维度下的扩展
expand维度扩展：返回自张量的一个新视图，该视图具有扩展到更大尺寸的单维度。参数为所需的扩展尺寸。展开一个张量并不分配新的内存，而是在现有张量上创建一个新的视图，其中大小为1的维度通过将stride设置为0扩展到更大的尺寸。大小为1的任何维度都可以扩展为任意值，而无需分配新内存。

（没有增加数据？？只改变了理解方式。）只能是由1扩张为N

unsqueeze维度增加只能使此维度下是0字节，不能使此维度变为多字节。


In [0]:
b=torch.rand(1,32,1,1)
b.shape

torch.Size([1, 32, 1, 1])

In [0]:
b.expand(4,32,14,14).shape #某维度下元素扩张，

torch.Size([4, 32, 14, 14])

In [0]:
b.expand(-1,-1,-1,-4).shape #-1表示扩张后不变，-4没有意义

torch.Size([1, 32, 1, -4])

In [0]:
b=torch.rand(1,2,1)
b

tensor([[[0.7190],
         [0.2480]]])

In [0]:
b.expand(1,2,3)

tensor([[[0.7190, 0.7190, 0.7190],
         [0.2480, 0.2480, 0.2480]]])

In [0]:
b

tensor([[[0.7190],
         [0.2480]]])

In [0]:
x=torch.tensor([[1], [2], [3]]) #torch.Size([3, 1])
x.expand(3, 4)

tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

### repeat 某维度下的复制
repeat复制：沿着指定的维数重复这个张量。与expand()不同，这个函数复制了张量的数据。参数是沿着每个维度重复这个张量的次数。

增加了数据。以前的内存会更改掉，数据会复制，不能用原来的数据，原来的内存下些，重复多次后内存变多，会重新申请一块内存空间复制这些数据出来，不建议用.



In [0]:
b=torch.rand(1,32,1,1)
b.shape

torch.Size([1, 32, 1, 1])

In [0]:
b.repeat(4,1,14,14).shape #某维度下元素复制，参数是复制倍数

torch.Size([4, 32, 14, 14])

In [0]:
b=torch.rand(1,2,1)
b

tensor([[[0.4636],
         [0.1190]]])

In [0]:
b.repeat(1,1,3)

tensor([[[0.4636, 0.4636, 0.4636],
         [0.1190, 0.1190, 0.1190]]])

In [0]:
b

tensor([[[0.4636],
         [0.1190]]])

In [0]:
x=torch.tensor([[1], [2], [3]]) #torch.Size([3, 1])
x.repeat(1, 4)

tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

### t 转置
.t() 方法只能适用于2D tensor（即矩阵），对3D和4D的不能使用

In [0]:
a=torch.randn(3,4)
a

tensor([[ 1.3874,  0.6938, -0.2808, -1.1140],
        [-1.1765, -0.4796,  0.5906,  0.5718],
        [-0.1357, -0.7222,  0.3665,  0.4756]])

In [0]:
a.t()

tensor([[ 1.3874, -1.1765, -0.1357],
        [ 0.6938, -0.4796, -0.7222],
        [-0.2808,  0.5906,  0.3665],
        [-1.1140,  0.5718,  0.4756]])

### transpose 维度两两交换
注意数据的维度顺序必须和存储顺序一致。

交换后经过多次变形如何再换回来：

In [0]:
a=torch.rand(4,3,32,32)

In [0]:
#一定把维度的先后顺序跟踪住，contiguous()打为连续值，再变形，最后变形为交换后的
#维度顺序(4,32,32,3)，就可以再交换回来了
a1=a.transpose(1,3).contiguous().view(4,3*32*32).view(4,32,32,3).transpose(1,3)

torch.all(torch.eq(a,a1)) #eq判断数据内容是否一致，all保证所有内容一致才返回1

tensor(1, dtype=torch.uint8)

In [0]:
a1=a.transpose(1,3).view(4,32,32,3).transpose(1,3) #维度交换后再换回来，view(4,32,32,3)可省
torch.all(torch.eq(a,a1))

tensor(1, dtype=torch.uint8)

交换举例：

In [0]:
a=torch.rand(1,2,3)
print(a.shape)
a

torch.Size([1, 2, 3])


tensor([[[0.7114, 0.4240, 0.7551],
         [0.5262, 0.4060, 0.7016]]])

In [0]:
a1=a.transpose(0,2) #交换第0、2维
print(a1.shape)
a1

torch.Size([3, 2, 1])


tensor([[[0.7114],
         [0.5262]],

        [[0.4240],
         [0.4060]],

        [[0.7551],
         [0.7016]]])

### permute 维度重排
重新排序维度

In [0]:
b=torch.rand(4,3,28,32)

In [0]:
b.permute(0,2,3,1).shape #重排维度，参数是以前的位置索引

torch.Size([4, 28, 32, 3])

### Broadcast 自动扩展维度
Broadcast 是自动的插入新的维度，并复制扩展新维度下shape。Expand 自动扩展维度，without copying data 但不用手动去复制数据，节省了内存消耗（相比于repeat）。

约定先从小维度匹配，大维度概念上更可能统一。指定了小维度（小维度要么为1将被扩展，要么与另一个tensor的shape相同），大维度就可以随意自动扩张。如要对[4,20,5]的小维度sorce上再加一组，[1]->[1,1,1]->[4,20,5]这种对小维度的5组sorce都做了相同元素的加法组sorce合法；[5]->[1,1,5]->[4,20,5]这种对小维度的5组sorce做了不同元素的加法合法；[4]->[1,1,4]这种不合法。

主要思想：先从前面插入1个维度（相当于unsqueeze操作）；再将新加入维度的size 1 扩张成相同的size（相当于expand、repeat操作）。

In [0]:
x=torch.arange(4).view(4,1)
y=torch.arange(3).view(1,3)
x,y

(tensor([[0],
         [1],
         [2],
         [3]]), tensor([[0, 1, 2]]))

In [0]:
a,b=torch.broadcast_tensors(x,y) #自动扩展x y的维度为torch.Size([4, 3])
a

tensor([[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [0]:
b

tensor([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]])

In [0]:
a+b

tensor([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]])